<a href="https://colab.research.google.com/github/victorantoniassi/DengAI_ds_course_project/blob/main/project_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [2]:
import pandas as pd
import numpy as np
from pycaret.regression import *
from pycaret.utils import check_metric

In [3]:
df_train = pd.read_csv('/datasets/dengue_features_train.csv')

In [4]:
df_labels = pd.read_csv('/datasets/dengue_labels_train.csv')

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1456 entries, 0 to 1455
Data columns (total 24 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   city                                   1456 non-null   object 
 1   year                                   1456 non-null   int64  
 2   weekofyear                             1456 non-null   int64  
 3   week_start_date                        1456 non-null   object 
 4   ndvi_ne                                1262 non-null   float64
 5   ndvi_nw                                1404 non-null   float64
 6   ndvi_se                                1434 non-null   float64
 7   ndvi_sw                                1434 non-null   float64
 8   precipitation_amt_mm                   1443 non-null   float64
 9   reanalysis_air_temp_k                  1446 non-null   float64
 10  reanalysis_avg_temp_k                  1446 non-null   float64
 11  rean

In [6]:
df_train.head(10)

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,297.742857,292.414286,299.8,295.9,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0
1,sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,298.442857,293.951429,300.9,296.4,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6
2,sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,298.878571,295.434286,300.5,297.3,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4
3,sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,299.228571,295.310000,301.4,297.0,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0
4,sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,299.664286,295.821429,301.9,297.5,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8
5,sj,1990,23,1990-06-04,NaN,0.174850,0.254314,0.181743,9.58,299.630000,299.764286,295.851429,302.4,298.1,26.49,79.891429,9.58,17.212857,2.100000,28.114286,6.942857,34.4,23.9,39.1
6,sj,1990,24,1990-06-11,0.112900,0.092800,0.205071,0.210271,3.48,299.207143,299.221429,295.865714,301.3,297.7,38.60,82.000000,3.48,17.234286,2.042857,27.414286,6.771429,32.2,23.3,29.7
7,sj,1990,25,1990-06-18,0.072500,0.072500,0.151471,0.133029,151.12,299.591429,299.528571,296.531429,300.6,298.4,30.00,83.375714,151.12,17.977143,1.571429,28.371429,7.685714,33.9,22.8,21.1
8,sj,1990,26,1990-06-25,0.102450,0.146175,0.125571,0.123600,19.32,299.578571,299.557143,296.378571,302.1,297.7,37.51,82.768571,19.32,17.790000,1.885714,28.328571,7.385714,33.9,22.8,21.1
9,sj,1990,27,1990-07-02,NaN,0.121550,0.160683,0.202567,14.41,300.154286,300.278571,296.651429,302.3,298.7,28.40,81.281429,14.41,18.071429,2.014286,28.328571,6.514286,33.9,24.4,1.1


In [7]:
df_train = pd.concat([df_train, df_labels['total_cases']], axis=1)

In [8]:
df_train = df_train.query('city == "sj"')

# Dataprep

In [9]:
#month extraction function
def extract_month(s):
    return int(s[5:7])

In [10]:
def feature_engineering(X):
    """
    Extracts the month out of date and converts it to a dummy variable
    """
    months = X.week_start_date.apply(extract_month)
 
    X.drop(['city', 'year', 'weekofyear', 'week_start_date', 'ndvi_ne'], axis=1, inplace=True)
    
    month_features = pd.get_dummies(months, prefix='m_')
    X = X.join(month_features)
    
    return X

In [11]:
X = feature_engineering(df_train)

In [12]:
X.head(10)

,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases,m__1,m__2,m__3,m__4,m__5,m__6,m__7,m__8,m__9,m__10,m__11,m__12
0,0.103725,0.198483,0.177617,12.42,297.572857,297.742857,292.414286,299.8,295.9,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0,4,0,0,0,1,0,0,0,0,0,0,0,0
1,0.142175,0.162357,0.155486,22.82,298.211429,298.442857,293.951429,300.9,296.4,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,5,0,0,0,0,1,0,0,0,0,0,0,0
2,0.172967,0.157200,0.170843,34.54,298.781429,298.878571,295.434286,300.5,297.3,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4,4,0,0,0,0,1,0,0,0,0,0,0,0
3,0.245067,0.227557,0.235886,15.36,298.987143,299.228571,295.310000,301.4,297.0,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0,3,0,0,0,0,1,0,0,0,0,0,0,0
4,0.262200,0.251200,0.247340,7.52,299.518571,299.664286,295.821429,301.9,297.5,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8,6,0,0,0,0,1,0,0,0,0,0,0,0
5,0.174850,0.254314,0.181743,9.58,299.630000,299.764286,295.851429,302.4,298.1,26.49,79.891429,9.58,17.212857,2.100000,28.114286,6.942857,34.4,23.9,39.1,2,0,0,0,0,0,1,0,0,0,0,0,0
6,0.092800,0.205071,0.210271,3.48,299.207143,299.221429,295.865714,301.3,297.7,38.60,82.000000,3.48,17.234286,2.042857,27.414286,6.771429,32.2,23.3,29.7,4,0,0,0,0,0,1,0,0,0,0,0,0
7,0.072500,0.151471,0.133029,151.12,299.591429,299.528571,296.531429,300.6,298.4,30.00,83.375714,151.12,17.977143,1.571429,28.371429,7.685714,33.9,22.8,21.1,5,0,0,0,0,0,1,0,0,0,0,0,0
8,0.146175,0.125571,0.123600,19.32,299.578571,299.557143,296.378571,302.1,297.7,37.51,82.768571,19.32,17.790000,1.885714,28.328571,7.385714,33.9,22.8,21.1,10,0,0,0,0,0,1,0,0,0,0,0,0
9,0.121550,0.160683,0.202567,14.41,300.154286,300.278571,296.651429,302.3,298.7,28.40,81.281429,14.41,18.071429,2.014286,28.328571,6.514286,33.9,24.4,1.1,6,0,0,0,0,0,0,1,0,0,0,0,0


In [13]:
#filling missing data using linear interpolation
X.interpolate(inplace=True)

In [14]:
#creating test dataset
X_train = X.sample(frac=0.9, random_state=42)
test = X.drop(X_train.index)

X_train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

print('Data for Modeling: ' + str(X_train.shape))
print('Unseen Data For Predictions ' + str(test.shape))

Data for Modeling: (842, 32)
Unseen Data For Predictions (94, 32)


In [15]:
#listing month columns
month_features = X.loc[:, X.columns.str.startswith("m__")].columns.tolist()

# Modelling

In [16]:
'''Setting up the experiment and the latest data preparation methods
Feature Selection Method -> Selects important features and reduces the feature space.
Feature selection is based on Random Forest, Light GBM and Correlation  
'''
exp1 = pycaret.regression.setup(X_train, target = 'total_cases', train_size = 0.7, numeric_features = month_features,
                                remove_multicollinearity = True, multicollinearity_threshold = 0.8,
                                feature_selection = True, log_experiment = True, experiment_name = 'model1', session_id=42)                 

,Description,Value
0,session_id,42
1,Target,total_cases
2,Original Data,"(842, 32)"
3,Missing Values,False
4,Numeric Features,31
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(589, 21)"


In [17]:
X_transformed = get_config('X')

In [18]:
X_transformed.head(10)

,m__12,m__10,reanalysis_tdtr_k,m__11,m__3,reanalysis_relative_humidity_percent,station_diur_temp_rng_c,m__6,m__5,ndvi_se,ndvi_sw,ndvi_nw,reanalysis_sat_precip_amt_mm,reanalysis_precip_amt_kg_per_m2,reanalysis_min_air_temp_k,m__1,m__9,station_precip_mm,m__4,m__2,m__8
0,0.0,0.0,1.971429,0.0,0.0,80.061432,6.371428,0.0,0.0,0.221783,0.183433,0.073750,6.010000,23.900000,297.500000,0.0,0.0,5.900000,0.0,0.0,0.0
1,0.0,0.0,2.128572,0.0,0.0,82.309998,7.542857,0.0,0.0,0.139800,0.162414,0.091675,65.900002,42.330002,298.399994,0.0,1.0,27.200001,0.0,0.0,0.0
2,0.0,0.0,2.671429,0.0,0.0,80.352859,7.700000,0.0,1.0,0.164414,0.132143,0.081750,68.000000,14.100000,297.000000,0.0,0.0,10.500000,0.0,0.0,0.0
3,1.0,0.0,1.771429,0.0,0.0,74.517143,6.642857,0.0,0.0,0.199300,0.171300,-0.072300,7.320000,7.450000,298.299988,0.0,0.0,40.599998,0.0,0.0,0.0
4,0.0,0.0,2.542857,0.0,0.0,78.928574,6.100000,0.0,0.0,0.208386,0.208557,0.025750,0.000000,5.800000,298.399994,0.0,0.0,13.000000,0.0,0.0,0.0
5,0.0,0.0,2.314286,0.0,1.0,76.878571,7.814286,0.0,0.0,0.191086,0.192857,0.182180,0.000000,5.500000,295.700012,0.0,0.0,2.900000,0.0,0.0,0.0
6,0.0,0.0,2.728571,0.0,0.0,74.798569,7.385714,0.0,0.0,0.194583,0.137500,0.027650,10.200000,5.350000,297.200012,0.0,0.0,4.100000,1.0,0.0,0.0
7,1.0,0.0,2.214286,0.0,0.0,76.127144,6.185714,0.0,0.0,0.137933,0.129756,0.123700,0.000000,19.299999,294.299988,0.0,0.0,13.700000,0.0,0.0,0.0
8,0.0,0.0,2.900000,0.0,0.0,78.604286,6.514286,0.0,0.0,0.290200,0.245514,0.189650,9.530000,23.180000,297.899994,0.0,0.0,22.700001,0.0,0.0,0.0
9,0.0,0.0,2.614286,0.0,0.0,80.935715,6.128572,0.0,0.0,0.142520,0.231700,0.029100,19.920000,29.200001,298.100006,0.0,0.0,14.300000,0.0,0.0,1.0


In [19]:
pipeline = pycaret.regression.get_config('prep_pipe')
pipeline

Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[], ml_usecase='regression',
                                      numerical_features=['m__1', 'm__2',
                                                          'm__3', 'm__4',
                                                          'm__5', 'm__6',
                                                          'm__7', 'm__8',
                                                          'm__9', 'm__10',
                                                          'm__11', 'm__12'],
                                      target='total_cases', time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strat...
                 Advanced_Feature_Selection_Classic(ml_usecase='regression',
                                              

In [20]:
#training and comparing regression models, all with default values for hyperparameters
metric = 'R2'
models1 = pycaret.regression.compare_models(sort = metric, fold = 5, turbo=False, errors="raise")

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,28.4908,2072.2011,45.3765,0.1927,1.1391,2.0819,0.158
gbr,Gradient Boosting Regressor,27.2672,2171.4690,46.1446,0.1750,1.0432,1.9142,0.142
ada,AdaBoost Regressor,34.8929,2169.9842,46.2145,0.1722,1.3383,3.7544,0.118
et,Extra Trees Regressor,25.9710,2202.3260,46.8176,0.1413,0.9808,1.7503,0.522
tr,TheilSen Regressor,24.2447,2289.9561,47.5803,0.1228,0.9634,1.5599,1.784
lr,Linear Regression,26.6462,2277.7858,47.5703,0.1174,1.0301,1.7604,0.574
ard,Automatic Relevance Determination,25.9362,2290.8082,47.6844,0.1142,1.0285,1.7652,0.076
ridge,Ridge Regression,25.7830,2323.3785,48.0219,0.1024,0.9864,1.6869,0.018
lar,Least Angle Regression,27.6618,2341.5228,48.2450,0.0912,1.0661,1.8486,0.022
br,Bayesian Ridge,26.6858,2417.5040,48.9678,0.0676,1.0671,2.0541,0.018


## Huber Regressor Model

In [21]:
#selecting and training a specific regressor
huber1 = pycaret.regression.create_model('huber', fold = 5)
huber1

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,19.9234,2384.0681,48.8269,0.0542,0.9399,1.0985
1,26.9363,3295.7612,57.4087,0.0311,0.9284,1.1118
2,20.4020,1939.7627,44.0427,0.0484,0.8838,1.0931
3,19.9914,1924.2406,43.8662,0.0460,0.8341,1.0104
4,23.3253,2960.0350,54.4062,0.0380,0.9429,0.9936
Mean,22.1157,2500.7735,49.7101,0.0435,0.9058,1.0615
SD,2.7187,548.3922,5.4474,0.0081,0.0417,0.0492


HuberRegressor(alpha=0.0001, epsilon=1.35, fit_intercept=True, max_iter=100,
               tol=1e-05, warm_start=False)

In [22]:
#hyperparameter optimization with Scikit-learn's RandomizedSearchCV function
tuned_huber1 = tune_model(huber1, n_iter = 100, optimize="MAE", fold = 5)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,19.7788,2428.0250,49.2750,0.0367,0.9240,1.0437
1,26.3633,3264.4461,57.1353,0.0403,0.9246,1.0118
2,20.6050,1965.2209,44.3308,0.0359,0.8908,1.0748
3,19.7914,1946.5081,44.1192,0.0349,0.8232,0.9574
4,23.5237,2979.8739,54.5882,0.0315,0.9725,1.0443
Mean,22.0124,2516.8148,49.8897,0.0359,0.9070,1.0264
SD,2.5736,531.1809,5.2755,0.0028,0.0493,0.0398


In [23]:
#predict on validation dataset
predict_model(tuned_huber1);

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Huber Regressor,23.6486,3131.4245,55.9591,0.0462,0.8898,1.0869


In [24]:
#cross-validation with all data(train dataset + validation dataset)
final_huber1 = finalize_model(tuned_huber1)
final_huber1

HuberRegressor(alpha=0.0001, epsilon=1.2, fit_intercept=True, max_iter=100,
               tol=1e-05, warm_start=False)

In [25]:
#predict on test dataset
test_predictions1 = predict_model(final_huber1, data=test)
test_predictions1.sample(10)

,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases,m__1,m__2,m__3,m__4,m__5,m__6,m__7,m__8,m__9,m__10,m__11,m__12,Label
40,0.095825,0.090271,0.099900,70.29,300.564286,300.735714,296.787143,303.2,298.5,26.00,80.084286,70.29,18.310000,2.642857,28.500000,7.300000,33.3,23.3,44.4,52,0,0,0,0,0,0,0,0,0,1,0,0,37.045121
22,0.050933,0.136929,0.119029,0.00,298.120000,298.314286,293.151429,300.0,296.8,4.40,73.951429,0.00,14.547143,2.400000,26.428571,7.771429,31.1,21.7,2.3,13,0,0,0,1,0,0,0,0,0,0,0,0,11.753176
55,0.161300,0.139883,0.095833,81.73,298.148571,298.164286,294.355714,299.9,296.5,32.30,79.607143,81.73,15.722857,1.900000,25.128571,5.314286,29.4,21.7,28.6,16,1,0,0,0,0,0,0,0,0,0,0,0,23.150001
72,0.032750,0.163443,0.110371,0.61,300.932857,301.100000,296.384286,303.2,299.2,6.30,76.435714,0.61,17.884286,2.857143,28.100000,6.042857,31.7,24.4,0.8,34,0,0,0,0,0,0,0,0,0,1,0,0,28.182765
0,0.142175,0.162357,0.155486,22.82,298.211429,298.442857,293.951429,300.9,296.4,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,5,0,0,0,0,1,0,0,0,0,0,0,0,6.912148
26,0.068650,0.122920,0.109960,0.00,297.175714,297.257143,293.444286,299.5,295.4,50.73,79.817143,0.00,14.852857,2.428571,25.242857,8.400000,30.6,20.0,51.7,20,0,0,1,0,0,0,0,0,0,0,0,0,22.805012
39,0.098233,0.143214,0.143414,13.18,300.420000,300.514286,296.385714,302.6,298.0,39.60,78.780000,13.18,17.828571,2.400000,28.400000,6.742857,32.2,23.9,25.4,34,0,0,0,0,0,0,0,0,1,0,0,0,31.514274
67,0.073267,0.079600,0.068071,73.11,301.061429,301.157143,297.128571,303.2,299.5,20.00,79.281429,73.11,18.674286,2.428571,27.771429,6.871429,33.3,22.8,72.7,7,0,0,0,0,0,0,0,0,0,1,0,0,34.789599
10,0.080850,0.134843,0.106800,0.00,297.218571,297.428571,293.578571,300.3,295.8,21.00,80.238571,0.00,14.968571,2.800000,26.200000,5.714286,32.2,22.2,2.3,29,0,0,1,0,0,0,0,0,0,0,0,0,7.972449
44,0.047133,0.082543,0.068933,10.42,300.337143,300.457143,296.288571,302.8,298.1,30.86,78.990000,10.42,17.715714,2.714286,28.385714,7.071429,33.3,23.9,62.5,150,0,0,0,0,0,0,1,0,0,0,0,0,20.848833


In [26]:
check_metric(test_predictions1.total_cases, test_predictions1.Label, 'MAE')

16.2398

In [27]:
check_metric(test_predictions1.total_cases, test_predictions1.Label, 'R2')

0.1437

## TheilSen Regressor Model

In [28]:
#selecting and training a specific regressor
ts1 = pycaret.regression.create_model('tr', fold = 5)
ts1

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,22.9595,2198.6403,46.8897,0.1277,1.0257,1.6696
1,28.6477,3081.1273,55.5079,0.0942,0.9898,1.6123
2,21.7470,1735.0844,41.6543,0.1488,0.9210,1.5713
3,23.6820,1847.6803,42.9847,0.0839,0.9385,1.5161
4,24.1874,2587.2483,50.8650,0.1591,0.9420,1.4304
Mean,24.2447,2289.9561,47.5803,0.1228,0.9634,1.5599
SD,2.3493,495.2098,5.1059,0.0295,0.0386,0.0820


TheilSenRegressor(copy_X=True, fit_intercept=True, max_iter=1000,
                  max_subpopulation=10000, n_jobs=-1, n_subsamples=None,
                  random_state=42, tol=0.001, verbose=False)

In [29]:
#hyperparameter optimization with Scikit-learn's RandomizedSearchCV function
tuned_ts1 = tune_model(ts1, n_iter = 100, optimize="MAE", fold = 5)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,22.4939,2231.0423,47.2339,0.1149,1.0253,1.6166
1,27.7350,3050.3242,55.2297,0.1033,0.9500,1.4680
2,21.6131,1763.1988,41.9905,0.1350,0.9186,1.5281
3,23.1880,1865.1462,43.1873,0.0753,0.9500,1.4316
4,23.7504,2620.9666,51.1954,0.1482,0.8978,1.2605
Mean,23.7561,2306.1356,47.7674,0.1153,0.9484,1.4610
SD,2.1138,479.1297,4.9411,0.0254,0.0433,0.1182


In [30]:
#predict on validation dataset
predict_model(tuned_ts1);

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,TheilSen Regressor,24.6462,2816.3745,53.0695,0.1421,0.8967,1.3416


In [31]:
#cross-validation with all data(train dataset + validation dataset)
final_ts1 = finalize_model(tuned_ts1)
final_ts1

TheilSenRegressor(copy_X=True, fit_intercept=False, max_iter=1000,
                  max_subpopulation=10000, n_jobs=-1, n_subsamples=None,
                  random_state=42, tol=0.001, verbose=False)

In [32]:
#predict on test dataset
test_predictions2 = predict_model(final_ts1, data=test)
test_predictions2.sample(10)

,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases,m__1,m__2,m__3,m__4,m__5,m__6,m__7,m__8,m__9,m__10,m__11,m__12,Label
40,0.095825,0.090271,0.099900,70.29,300.564286,300.735714,296.787143,303.2,298.5,26.00,80.084286,70.29,18.310000,2.642857,28.500000,7.300000,33.3,23.3,44.4,52,0,0,0,0,0,0,0,0,0,1,0,0,59.639462
22,0.050933,0.136929,0.119029,0.00,298.120000,298.314286,293.151429,300.0,296.8,4.40,73.951429,0.00,14.547143,2.400000,26.428571,7.771429,31.1,21.7,2.3,13,0,0,0,1,0,0,0,0,0,0,0,0,11.982174
55,0.161300,0.139883,0.095833,81.73,298.148571,298.164286,294.355714,299.9,296.5,32.30,79.607143,81.73,15.722857,1.900000,25.128571,5.314286,29.4,21.7,28.6,16,1,0,0,0,0,0,0,0,0,0,0,0,27.227633
72,0.032750,0.163443,0.110371,0.61,300.932857,301.100000,296.384286,303.2,299.2,6.30,76.435714,0.61,17.884286,2.857143,28.100000,6.042857,31.7,24.4,0.8,34,0,0,0,0,0,0,0,0,0,1,0,0,48.180260
0,0.142175,0.162357,0.155486,22.82,298.211429,298.442857,293.951429,300.9,296.4,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,5,0,0,0,0,1,0,0,0,0,0,0,0,14.818694
26,0.068650,0.122920,0.109960,0.00,297.175714,297.257143,293.444286,299.5,295.4,50.73,79.817143,0.00,14.852857,2.428571,25.242857,8.400000,30.6,20.0,51.7,20,0,0,1,0,0,0,0,0,0,0,0,0,22.771070
39,0.098233,0.143214,0.143414,13.18,300.420000,300.514286,296.385714,302.6,298.0,39.60,78.780000,13.18,17.828571,2.400000,28.400000,6.742857,32.2,23.9,25.4,34,0,0,0,0,0,0,0,0,1,0,0,0,49.720524
67,0.073267,0.079600,0.068071,73.11,301.061429,301.157143,297.128571,303.2,299.5,20.00,79.281429,73.11,18.674286,2.428571,27.771429,6.871429,33.3,22.8,72.7,7,0,0,0,0,0,0,0,0,0,1,0,0,53.178134
10,0.080850,0.134843,0.106800,0.00,297.218571,297.428571,293.578571,300.3,295.8,21.00,80.238571,0.00,14.968571,2.800000,26.200000,5.714286,32.2,22.2,2.3,29,0,0,1,0,0,0,0,0,0,0,0,0,14.673233
44,0.047133,0.082543,0.068933,10.42,300.337143,300.457143,296.288571,302.8,298.1,30.86,78.990000,10.42,17.715714,2.714286,28.385714,7.071429,33.3,23.9,62.5,150,0,0,0,0,0,0,1,0,0,0,0,0,22.357773


In [33]:
check_metric(test_predictions2.total_cases, test_predictions2.Label, 'MAE')

17.4528

In [34]:
check_metric(test_predictions2.total_cases, test_predictions2.Label, 'R2')

0.181

## Gradient Boosting Regressor Model

In [35]:
#selecting and training a specific regressor
gbr1 = pycaret.regression.create_model('gbr', fold = 5)
gbr1

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,26.0715,1844.9726,42.9531,0.2680,1.1235,1.9195
1,31.0062,3176.2849,56.3585,0.0662,1.0261,1.8655
2,22.4920,1395.7695,37.3600,0.3153,0.9531,1.7626
3,27.6874,1942.0663,44.0689,0.0371,1.0814,2.0198
4,29.0788,2498.2518,49.9825,0.1881,1.0320,2.0037
Mean,27.2672,2171.4690,46.1446,0.1750,1.0432,1.9142
SD,2.8869,612.7605,6.4918,0.1089,0.0573,0.0944


GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=42, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [36]:
#hyperparameter optimization with Scikit-learn's RandomizedSearchCV function
tuned_gbr1 = tune_model(gbr1, n_iter = 100, optimize="R2", fold = 5)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,27.2746,2150.3553,46.3719,0.1469,1.1427,2.1398
1,29.4549,2809.3700,53.0035,0.1741,1.0264,1.8729
2,23.8231,1536.9951,39.2045,0.2460,1.0023,1.8672
3,26.5655,1607.4890,40.0935,0.2030,1.0705,1.9954
4,24.8209,1767.7309,42.0444,0.4255,0.9495,1.5518
Mean,26.3878,1974.3881,44.1436,0.2391,1.0383,1.8854
SD,1.9625,468.3576,5.0728,0.0988,0.0652,0.1942


In [37]:
#predict on validation dataset
predict_model(tuned_gbr1);

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Gradient Boosting Regressor,28.3782,2090.0155,45.7167,0.3634,1.0543,1.9937


In [38]:
#cross-validation with all data(train dataset + validation dataset)
final_gbr1 = finalize_model(tuned_gbr1)
final_gbr1

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=9,
                          max_features=1.0, max_leaf_nodes=None,
                          min_impurity_decrease=0.0005, min_impurity_split=None,
                          min_samples_leaf=3, min_samples_split=4,
                          min_weight_fraction_leaf=0.0, n_estimators=40,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=42, subsample=0.7, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [39]:
#predict on test dataset
test_predictions3 = predict_model(final_gbr1, data=test)
test_predictions3.sample(10)

,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases,m__1,m__2,m__3,m__4,m__5,m__6,m__7,m__8,m__9,m__10,m__11,m__12,Label
40,0.095825,0.090271,0.099900,70.29,300.564286,300.735714,296.787143,303.2,298.5,26.00,80.084286,70.29,18.310000,2.642857,28.500000,7.300000,33.3,23.3,44.4,52,0,0,0,0,0,0,0,0,0,1,0,0,12.218757
22,0.050933,0.136929,0.119029,0.00,298.120000,298.314286,293.151429,300.0,296.8,4.40,73.951429,0.00,14.547143,2.400000,26.428571,7.771429,31.1,21.7,2.3,13,0,0,0,1,0,0,0,0,0,0,0,0,21.361252
55,0.161300,0.139883,0.095833,81.73,298.148571,298.164286,294.355714,299.9,296.5,32.30,79.607143,81.73,15.722857,1.900000,25.128571,5.314286,29.4,21.7,28.6,16,1,0,0,0,0,0,0,0,0,0,0,0,18.342011
72,0.032750,0.163443,0.110371,0.61,300.932857,301.100000,296.384286,303.2,299.2,6.30,76.435714,0.61,17.884286,2.857143,28.100000,6.042857,31.7,24.4,0.8,34,0,0,0,0,0,0,0,0,0,1,0,0,98.609642
0,0.142175,0.162357,0.155486,22.82,298.211429,298.442857,293.951429,300.9,296.4,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,5,0,0,0,0,1,0,0,0,0,0,0,0,16.809558
26,0.068650,0.122920,0.109960,0.00,297.175714,297.257143,293.444286,299.5,295.4,50.73,79.817143,0.00,14.852857,2.428571,25.242857,8.400000,30.6,20.0,51.7,20,0,0,1,0,0,0,0,0,0,0,0,0,29.826308
39,0.098233,0.143214,0.143414,13.18,300.420000,300.514286,296.385714,302.6,298.0,39.60,78.780000,13.18,17.828571,2.400000,28.400000,6.742857,32.2,23.9,25.4,34,0,0,0,0,0,0,0,0,1,0,0,0,70.908413
67,0.073267,0.079600,0.068071,73.11,301.061429,301.157143,297.128571,303.2,299.5,20.00,79.281429,73.11,18.674286,2.428571,27.771429,6.871429,33.3,22.8,72.7,7,0,0,0,0,0,0,0,0,0,1,0,0,39.114022
10,0.080850,0.134843,0.106800,0.00,297.218571,297.428571,293.578571,300.3,295.8,21.00,80.238571,0.00,14.968571,2.800000,26.200000,5.714286,32.2,22.2,2.3,29,0,0,1,0,0,0,0,0,0,0,0,0,13.680650
44,0.047133,0.082543,0.068933,10.42,300.337143,300.457143,296.288571,302.8,298.1,30.86,78.990000,10.42,17.715714,2.714286,28.385714,7.071429,33.3,23.9,62.5,150,0,0,0,0,0,0,1,0,0,0,0,0,17.445497


In [40]:
check_metric(test_predictions3.total_cases, test_predictions3.Label, 'MAE')

18.8949

In [41]:
check_metric(test_predictions3.total_cases, test_predictions3.Label, 'R2')

0.1473

## Light Gradient Boosting Machine Model

In [42]:
#selecting and training a specific regressor
lgbm1 = pycaret.regression.create_model('lightgbm', fold = 5)
lgbm1

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,28.1289,2156.4253,46.4373,0.1445,1.2020,2.2803
1,30.4550,2578.0057,50.7741,0.2421,1.0734,1.8498
2,25.1841,1561.2608,39.5128,0.2341,1.0879,1.9360
3,30.1164,1967.7039,44.3588,0.0244,1.2208,2.4002
4,28.5694,2097.6099,45.7997,0.3183,1.1116,1.9433
Mean,28.4908,2072.2011,45.3765,0.1927,1.1391,2.0819
SD,1.8747,327.2789,3.6292,0.1006,0.0605,0.2168


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [43]:
#hyperparameter optimization with Scikit-learn's RandomizedSearchCV function
tuned_lgbm1 = tune_model(lgbm1, n_iter = 100, optimize="R2", fold = 5)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,25.1109,1718.8788,41.4594,0.3181,1.1227,2.0049
1,28.4875,2484.8883,49.8487,0.2695,1.0284,1.6951
2,24.5971,1589.5817,39.8696,0.2202,1.0306,1.7336
3,29.4957,1904.6885,43.6427,0.0557,1.1618,2.3833
4,28.5507,2242.9759,47.3601,0.2710,1.1673,2.0515
Mean,27.2484,1988.2026,44.4361,0.2269,1.1022,1.9737
SD,1.9940,331.6669,3.6929,0.0910,0.0613,0.2490


In [44]:
#predict on validation dataset
predict_model(tuned_lgbm1);

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,26.4902,1849.2951,43.0034,0.4367,1.0092,1.592


In [45]:
#cross-validation with all data(train dataset + validation dataset)
final_lgbm1 = finalize_model(tuned_lgbm1)
final_lgbm1

LGBMRegressor(bagging_fraction=0.7, bagging_freq=0, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=0.4,
              importance_type='split', learning_rate=0.15, max_depth=-1,
              min_child_samples=16, min_child_weight=0.001, min_split_gain=0.2,
              n_estimators=140, n_jobs=-1, num_leaves=4, objective=None,
              random_state=42, reg_alpha=0.4, reg_lambda=4, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [46]:
#predict on test dataset
test_predictions4 = predict_model(final_lgbm1, data=test)
test_predictions4.sample(10)

,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases,m__1,m__2,m__3,m__4,m__5,m__6,m__7,m__8,m__9,m__10,m__11,m__12,Label
40,0.095825,0.090271,0.099900,70.29,300.564286,300.735714,296.787143,303.2,298.5,26.00,80.084286,70.29,18.310000,2.642857,28.500000,7.300000,33.3,23.3,44.4,52,0,0,0,0,0,0,0,0,0,1,0,0,53.476896
22,0.050933,0.136929,0.119029,0.00,298.120000,298.314286,293.151429,300.0,296.8,4.40,73.951429,0.00,14.547143,2.400000,26.428571,7.771429,31.1,21.7,2.3,13,0,0,0,1,0,0,0,0,0,0,0,0,25.836902
55,0.161300,0.139883,0.095833,81.73,298.148571,298.164286,294.355714,299.9,296.5,32.30,79.607143,81.73,15.722857,1.900000,25.128571,5.314286,29.4,21.7,28.6,16,1,0,0,0,0,0,0,0,0,0,0,0,16.113505
72,0.032750,0.163443,0.110371,0.61,300.932857,301.100000,296.384286,303.2,299.2,6.30,76.435714,0.61,17.884286,2.857143,28.100000,6.042857,31.7,24.4,0.8,34,0,0,0,0,0,0,0,0,0,1,0,0,48.242631
0,0.142175,0.162357,0.155486,22.82,298.211429,298.442857,293.951429,300.9,296.4,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,5,0,0,0,0,1,0,0,0,0,0,0,0,18.117504
26,0.068650,0.122920,0.109960,0.00,297.175714,297.257143,293.444286,299.5,295.4,50.73,79.817143,0.00,14.852857,2.428571,25.242857,8.400000,30.6,20.0,51.7,20,0,0,1,0,0,0,0,0,0,0,0,0,21.478263
39,0.098233,0.143214,0.143414,13.18,300.420000,300.514286,296.385714,302.6,298.0,39.60,78.780000,13.18,17.828571,2.400000,28.400000,6.742857,32.2,23.9,25.4,34,0,0,0,0,0,0,0,0,1,0,0,0,39.859869
67,0.073267,0.079600,0.068071,73.11,301.061429,301.157143,297.128571,303.2,299.5,20.00,79.281429,73.11,18.674286,2.428571,27.771429,6.871429,33.3,22.8,72.7,7,0,0,0,0,0,0,0,0,0,1,0,0,81.327668
10,0.080850,0.134843,0.106800,0.00,297.218571,297.428571,293.578571,300.3,295.8,21.00,80.238571,0.00,14.968571,2.800000,26.200000,5.714286,32.2,22.2,2.3,29,0,0,1,0,0,0,0,0,0,0,0,0,10.877441
44,0.047133,0.082543,0.068933,10.42,300.337143,300.457143,296.288571,302.8,298.1,30.86,78.990000,10.42,17.715714,2.714286,28.385714,7.071429,33.3,23.9,62.5,150,0,0,0,0,0,0,1,0,0,0,0,0,9.003267


In [47]:
check_metric(test_predictions4.total_cases, test_predictions4.Label, 'MAE')

16.2444

In [48]:
check_metric(test_predictions4.total_cases, test_predictions4.Label, 'R2')

0.3375

# Evaluation

In [49]:
#Huber Regressor

'''
metrics results with the validation dataset
MAE = 23.6486
R2 = 0.0462

metrics results with the test dataset
MAE = 16.2398
R2 = 0.1437
'''

evaluate_model(tuned_huber1)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [58]:
save_model(final_huber1,'Final Huber Model 30-04-2021')

Transformation Pipeline and Model Succesfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=['m__1', 'm__2',
                                                           'm__3', 'm__4',
                                                           'm__5', 'm__6',
                                                           'm__7', 'm__8',
                                                           'm__9', 'm__10',
                                                           'm__11', 'm__12'],
                                       target='total_cases', time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strat...
                                                     top_features_to_pick=0.19999999999999996)),
            

In [51]:
#Light Gradient Boosting Machine

'''
metrics results with the validation dataset
MAE = 26.4902
R2 = 0.4367

metrics results with the test dataset
MAE = 16.2444
R2 = 0.3375
'''

evaluate_model(tuned_lgbm1)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [57]:
save_model(final_lgbm1,'Final LightGBM Model 30-04-2021')

Transformation Pipeline and Model Succesfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=['m__1', 'm__2',
                                                           'm__3', 'm__4',
                                                           'm__5', 'm__6',
                                                           'm__7', 'm__8',
                                                           'm__9', 'm__10',
                                                           'm__11', 'm__12'],
                                       target='total_cases', time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strat...
                                boosting_type='gbdt', class_weight=None,
                                cols